This notebook investigates the descrepancies between H&Y values reported in 'PPMI_Original_Cohort_BL_to_Year_5_Dataset_Apr2020.csv' (variables 'hy' and 'hy_on) and in 'MDS_UPDRS_Part_III.csv' (variable NHY).

In [1]:
# Download required files

import os
import ppmi_downloader

data_dir = 'data'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)
required_files = ['PPMI_Original_Cohort_BL_to_Year_5_Dataset_Apr2020.csv', 'MDS_UPDRS_Part_III.csv']
missing_files = [x for x in required_files if not os.path.exists(os.path.join(data_dir, x))]


if len(missing_files) > 0:
    ppmi = ppmi_downloader.PPMIDownloader('', '')
    ppmi.download_metadata(missing_files, destination_dir=data_dir, headless=False, timeout=600)


In [2]:
import os.path as op
import pandas as pd

updrs = pd.read_csv(op.join('data','MDS_UPDRS_Part_III.csv'))
keep = ['PATNO', 'EVENT_ID', 'PDSTATE', 'NHY', 'PDMEDDT']
updrs = updrs.drop([x for x in updrs.columns if x not in keep], axis=1)

# Focus on PDSTATE OFF
#updrs = updrs[updrs['PDSTATE']!='ON']

curated = pd.read_csv(op.join('data', 'PPMI_Original_Cohort_BL_to_Year_5_Dataset_Apr2020.csv'))
keep = ['PATNO', 'EVENT_ID', 'hy', 'hy_on', 'visit_date']
curated = curated.drop([x for x in curated.columns if x not in keep], axis=1)

/usr/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (10,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df = updrs.merge(curated, how='outer', on=['PATNO', 'EVENT_ID'])

In [4]:
# How many values are missing in UPDRS-III?
missing_NHY = df[(df['hy'].notnull()) & (df['NHY'].isnull())]
len(missing_NHY)

43

Let's ignore these missing values for now, as they aren't so numerous.

In [5]:
# How many values are missing in Cohort Cut?
missing_hy = df[(df['hy'].isnull()) & (df['NHY'].notnull())]
len(missing_hy)

14332

Most of these missing values, but not all of them, are due to the fact that the Cohort Cut only covers the first 5 years of the study.